In [1]:
%load_ext autoreload
%autoreload 2

In [95]:
%matplotlib widget
from ipywidgets import *
import numpy as np
import k3d
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar, brentq, minimize
from sloppy.optic import *
from sloppy.raytracing import *
from sloppy.abcd import *
from sloppy.utils import *
from sloppy.tools import *

In [3]:
from cavities import *

In [4]:
coeffc2 = lambda x: 1./(2*x)
coeffc4 = lambda x: 1./(8*x**3)
coeffc6 = lambda x: 1./(16*x**5)
coeffc8 = lambda x: 5./(128*x**7)

In [5]:
print(coeffc4(5.0), coeffc6(5.0), coeffc8(5.0))

0.001 2e-05 5e-07


In [102]:
def LensCavAsphere(arm1=42., arm2=21., base=17., angle=110., lens_dist=24.47, c4=coeffc4(5.0), c6=coeffc6(5.0), c8=coeffc8(5.0)):
    angle = np.deg2rad(angle)
    Rlens=5.0
    lens_diam=6.35
    lens_thick=4.
    
    eps = 0. #1e-4 #hack
    p0 = np.array([lens_dist/2.-lens_thick/2.,0,0])
    p1 = np.array([lens_dist/2.+lens_thick/2.,0,0])
    p2 = np.array([arm1/2.,0,0])
    p3 = np.array([np.cos(angle)*arm2/2.,base,np.sin(angle)*arm2/2.])
    p4 = np.array([-np.cos(angle)*arm2/2.,base,-np.sin(angle)*arm2/2.])
    p5 = np.array([-arm1/2.,0,0])
    p6 = np.array([-lens_dist/2.-lens_thick/2.,0,0])
    p7 = np.array([-lens_dist/2.+lens_thick/2.,0,0])
    ps = np.stack([p0,p1,p2,p3,p4,p5,p6,p7], axis=0)
    
    geom = geometry(ps)
    ns = geom['refl']
    ps = geom['mir']
    angles = geom['angles']
    Rtr = geom['R']
    ax_x = geom['xin']
    ax_y = np.cross(ns, ax_x)
        
    hi = 12.7
    qi=7.75
    ng = 1.4537
    
    coef = np.zeros(9)
    coef[2] = coeffc2(Rlens)
    coef[4] = c4
    coef[6] = c6
    coef[8] = c8
     #negative sign of firstt cuved surface for abcd matrix
    elements = [Glass(p=ps[0], n=ns[0], ax=ax_x[0], ay=ax_y[0], Rbasis=Rtr[0], diameter=lens_diam, n2=ng),\
                FreeFormInterface(p=ps[1], n=ns[1], ax=ax_x[1], ay=ax_y[1], Rbasis=Rtr[1], diameter=lens_diam, n1=ng, coef=coef),\
                Mirror(p=ps[2], n=ns[2], ax=ax_x[2], ay=ax_y[2], Rbasis=Rtr[2], diameter=hi),\
                Mirror(p=ps[3], n=ns[3], ax=ax_x[3], ay=ax_y[3], Rbasis=Rtr[3], diameter=hi),\
                Mirror(p=ps[4], n=ns[4], ax=ax_x[4], ay=ax_y[4], Rbasis=Rtr[4], diameter=hi),\
                Mirror(p=ps[5], n=ns[5], ax=ax_x[5], ay=ax_y[5], Rbasis=Rtr[5], diameter=hi),\
                FreeFormInterface(p=ps[6], n=ns[6], ax=ax_x[6], ay=ax_y[6], Rbasis=Rtr[6], diameter=lens_diam, n2=ng, coef=-coef),\
                Glass(p=ps[7], n=ns[7], ax=ax_x[7], ay=ax_y[7], Rbasis=Rtr[7], diameter=lens_diam, n1=ng)]
    return elements

In [7]:
magnitude = lambda x: int(np.floor(np.log10(x)))

In [66]:
cavfct = LensCavAsphere
lo = Layout(width='80%', height='30px')
sliders = {v.name: FloatSlider(value=v.default, min=v.default*0.5, max=v.default*1.5, step=10**(magnitude(v.default)-2), readout_format='.2e', layout=lo) for v in inspect.signature(cavfct).parameters.values()}

In [67]:
def cavity_parameter_interaction_factory(cavfct, parname, scanrange, N = 300):
    fig, ax = plt.subplots(ncols=2, figsize=(8,4), sharex=True)
    lws = [ax[0].plot([0, 1], [0, 1])[0] for i in range(2)]
    ax[0].set_ylabel('um')
    lfs = [ax[1].plot([0, 1], [0, 1])[0] for i in range(4)]
    ax[1].set_ylabel('MHz')
    axh1 = ax[1].axhline(0, color='grey')
    axv0 = ax[0].axvline(0, color='grey')
    axv1 = ax[1].axvline(0, color='grey')
    plt.show()
    
    magnitude = lambda x: int(np.floor(np.log10(x)))
    
    def update_waists_vs_params(cavfct, parname, scanrange, N = 300, **kwargs):
        stab = lambda m: abs(0.5*np.trace(m))<1
        #La = inspect.signature(cavfct).parameters[parname].default
        La = kwargs[parname]
        Las = La + np.linspace(-scanrange*La, scanrange*La, N)
        ms = np.zeros((N))
        ws = np.zeros((N,2))
        freqs = np.zeros((N,4))
        for i, l in enumerate(Las):
            pardct = kwargs.copy()
            pardct.update({parname: l})
            sys = RaySystem( cavfct(**pardct) )

            try:
                system = sys.abcd
                w = system.waist_at(0)
            except:
                ws[i,:] = np.nan
                ms[i] = np.nan
                freqs[i,...] = np.nan
            else:
                ms[i] = stab(system.abcd_rt)
                ws[i,:] = np.sort(w)
                freqs[i,...] = np.concatenate(system.get_freqs())
                #ft, fs3 = system.get_freqs()
                #fsr = system.fsr
                #freqs[i,...] = np.concatenate((ft, np.mod(3*ft, fsr)))
                idx = np.argmin(np.abs(freqs[:,2]))
        for i in range(2):
            lws[i].set_data(np.stack((Las, ws[:,i]*1e3), axis=0))
        for i in range(4):
            lfs[i].set_data(np.stack((Las, freqs[:,i]*1e-6), axis=0))
        axv0.set_xdata([Las[idx], Las[idx]])
        axv1.set_xdata([Las[idx], Las[idx]])
        Ldeg = Las[idx]
        ax[0].set_xlim(Las[0], Las[-1])
        ax[0].set_ylim(1.1*np.nanmin(ws*1e3), 1.1*np.nanmax(ws*1e3))
        ax[1].set_ylim(1.1*np.nanmin(freqs*1e-6), 1.1*np.nanmax(freqs*1e-6))
        fig.canvas.draw_idle()

    lo = Layout(width='80%', height='30px')
    sliders = {v.name: FloatSlider(value=v.default, min=v.default*0.5, max=v.default*1.5, step=10**(magnitude(v.default)-2), readout_format='.2e', layout=lo) for v in inspect.signature(cavfct).parameters.values()}
    #sliders.update({'scanrange': FloatSlider(value=scanrange, min=scanrange*1e-1, max=scanrange*1e-1, step=10**(magnitude(scanrange)-2), readout_format='.1e', layout=lo)})
    sliders.update({'scanrange': FloatLogSlider(value=scanrange, min=-3, max=1, step=0.5, layout=lo)})
    return interactive(update_waists_vs_params, cavfct=fixed(cavfct), parname = fixed('lens_dist'), N=fixed(N), **sliders)

In [18]:
cavity_parameter_interaction_factory(LensCavAsphere, 'lens_dist', 1e-1, N=100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatLogSlider(value=0.1, description='scanrange', layout=Layout(height='30px', width='8…

In [25]:
def waists_vs_params(cavfct, parname, scanrange, N = 300, **kwargs):
    stab = lambda m: abs(0.5*np.trace(m))<1
    #La = inspect.signature(cavfct).parameters[parname].default
    La = kwargs[parname]
    Las = La + np.linspace(-scanrange*La, scanrange*La, N)
    ms = np.zeros((N))
    ws = np.zeros((N,2))
    freqs = np.zeros((N,4))
    for i, l in enumerate(Las):
        pardct = kwargs.copy()
        pardct.update({parname: l})
        sys = RaySystem( cavfct(**pardct) )

        try:
            system = sys.abcd
            w = system.waist_at(0)
        except:
            ws[i,:] = np.nan
            ms[i] = np.nan
            freqs[i,...] = np.nan
        else:
            ms[i] = stab(system.abcd_rt)
            ws[i,:] = np.sort(w)
            freqs[i,...] = np.concatenate(system.get_freqs())

    idx = np.argmin(np.abs(freqs[:,2]))
    g, ax = plt.subplots(ncols=2, figsize=(8,4))
    ax[0].plot(Las, ws*1e3)
    ax[0].set_ylabel('um')
    ax[1].plot(Las, freqs*1e-6)
    ax[1].set_ylabel('MHz')
    ax[1].axhline(0, color='grey')
    ax[0].axvline(Las[idx], color='grey')
    ax[1].axvline(Las[idx], color='grey')
    plt.show()
    Ldeg = Las[idx]
    return Ldeg

In [103]:
waists_vs_param(LensCavAsphere, 'lens_dist', 1e-1)

res = degeneracy_length(LensCavAsphere, 'lens_dist', 1e-1)
Ldeg = res.x
print(res)

c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\abcd.py:184: RuntimeWarning: invalid value encountered in sqrt
  ws = np.sqrt(lam/(-np.pi*di.imag*n))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

     fun: 105952.29785716636
 message: 'Solution found.'
    nfev: 10
  status: 0
 success: True
       x: 24.470978485807766


In [20]:
elements = LensCavAsphere(lens_dist=Ldeg)
sys = RaySystem(elements)

In [21]:
system = sys.abcd
print(system.Ltot)
system.waist_at(0)

125.96287424209984


array([0.01974093, 0.01974093])

In [22]:
x = np.linspace(0, system.Ltot, 500)
ws = system.compute_waists(x)

plt.figure()
plt.plot(x, ws)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
mu = np.array([0., 0., 0., 0.])
ray = sys.screen.eigenvectors_to_rays(mu)

In [14]:
try:
    plot.close()
except:
    pass
plot = k3d.plot(camera_auto_fit=True, antialias=True)

for i, el in enumerate(sys.elements):
    plot += el.plot(opacity=0.4)

for i, el in enumerate(sys.elements):
    plot_element_ax(el, plot, length=5.)

plot_rays(ray, plot)
plot.display()

C:\ProgramData\Anaconda3\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [21]:
def rays_interaction_factory(cavfct, parname, scanrange=1e-2):
    fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(11,6.6), sharex='row', sharey='row')
    ms = 8.
    lines = [a.scatter([], [], c=[], cmap='jet', s=ms) for a in ax.flatten()]
    ax[0,0].set_title('Raytracing')
    ax[0,1].set_title('ABCD')
    ax[0,2].set_title('ABCD (Extracted)')
    plt.xlabel('x [mm]')
    plt.ylabel('y [mm]')

    res = degeneracy_length(cavfct, parname, scanrange)
    Ldeg = res.x

    def makeidx(hit_m):
        return np.arange(hit_m.shape[0])
    
    magnitude = lambda x: int(np.floor(np.log10(x)))
    
    def update(ar=0.0, br=0.0, ap=0., bp=0., Nrt=500, **kwargs):
        #kwargs.update({parname: Ldeg+dl*1e-1})
        elements = cavfct(**kwargs)
        sys = RaySystem(elements)
        system = sys.abcd
        mu1, mu2 = system.q
        waist = system.waist_at(0)[0] #fix for now since waist is not at 0 anymore!
        rmu1 = np.linalg.norm(np.real(mu1[:2]))
        rmu2 = np.linalg.norm(np.real(mu2[:2]))
        mu = np.real(ar*waist/rmu1*mu1*np.exp(1j*ap) + br*waist/rmu2*mu2*np.exp(1j*bp))
        
        ray0 = sys.screen.eigenvectors_to_rays(mu)
 
        traj_hit = sys.propagate(ray0, Nrt=Nrt, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
        hit_scrs = sys.screen.s_to_screen_coords(traj_hit[:,1,0,:])

        lines[0].set_offsets(hit_scr[:,:])
        lines[0].set_array(makeidx(hit_scr))
        lines[3].set_offsets(hit_scrs[:,:])
        lines[3].set_array(makeidx(hit_scrs))
        
        hit_m = propagate_ABCD(mu, system.abcd_rt, Nrt=Nrt)
        lines[1].set_offsets(hit_m[:,0:2])
        lines[1].set_array(makeidx(hit_m))
        lines[4].set_offsets(hit_m[:,2:])
        lines[4].set_array(makeidx(hit_m))
        
        #abcd_fd = sys.extract_ABCD(epsr=1e-3, epss=1e-3, Nrt=1) 
        #hit_fd = propagate_ABCD(mu, abcd_fd, Nrt=Nrt)
        #lines[2].set_offsets(hit_fd[:,0:2])
        #lines[2].set_array(makeidx(hit_fd))
        #lines[5].set_offsets(hit_fd[:,2:])
        #lines[5].set_array(makeidx(hit_fd))
        
        for l in lines:
            l.set_clim(vmin=0, vmax=Nrt)
            
        ax[0,0].set_xlim(1.5*np.nanmin(hit_scr[:,0]), 1.5*np.nanmax(hit_scr[:,0]))
        ax[0,0].set_ylim(1.5*np.nanmin(hit_scr[:,1]), 1.5*np.nanmax(hit_scr[:,1]))
        ax[1,0].set_xlim(1.5*np.nanmin(hit_scrs[:,0]), 1.5*np.nanmax(hit_scrs[:,0]))
        ax[1,0].set_ylim(1.5*np.nanmin(hit_scrs[:,1]), 1.5*np.nanmax(hit_scrs[:,1]))
        fig.canvas.draw_idle()

    lo = Layout(width='80%', height='30px')

    sliders = {v.name: FloatSlider(value=v.default, min=v.default*0.5, max=v.default*1.5, step=10**(magnitude(v.default)-2), readout_format='.2e', layout=lo) for v in inspect.signature(cavfct).parameters.values()}
    ars = FloatSlider(value=0., min=0., max=6., step=1e-2, readout_format='.3f', layout=lo)
    brs = FloatSlider(value=0.1, min=0., max=6., step=1e-2, readout_format='.3f', layout=lo)
    aps = FloatSlider(value=0.1, min=0., max=1*np.pi, step=1e-2, readout_format='.3f', layout=lo)
    bps = FloatSlider(value=0., min=0., max=1*np.pi, step=1e-2, readout_format='.3f', layout=lo)
    Nrts = IntSlider(value=100, min=100, max=2000, step=100, layout=lo)
    raysliders = {'ar': ars, 'br': brs, 'ap': aps, 'bp': bps, 'Nrt': Nrts}
    sliders.update(**raysliders)
    return interactive(update, **sliders)

In [91]:
rays_interaction_factory(LensCavAsphere, 'lens_dist', rmax=10.)

c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\tools.py:121: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(11,6.6), sharex='row', sharey='row')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.1, description='ar', layout=Layout(height='30px', width='80%'), max=…

In [25]:
cavfct = LensCavAsphere
parname = 'lens_dist'
res = degeneracy_length(cavfct, parname, 1e-3)
Ldeg = res.x
print(res)
elements = cavfct(**{parname: Ldeg})
sys = RaySystem(elements)
system = sys.abcd
mu1, mu2 = system.q

     fun: 152378.3900458578
 message: 'Solution found.'
    nfev: 6
  status: 0
 success: True
       x: 24.470976400936046


In [26]:
br = 0.01
mu = np.real(br*mu1)
ray0 = sys.screen.eigenvectors_to_rays(mu)

traj_hit = sys.propagate(ray0, Nrt=30, at_screen=True)
hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])

In [28]:
def makeidx(hit_m):
    return np.arange(hit_m.shape[0])
plt.figure()
hit_1 = hit_scr[::3,:]
plt.scatter(hit_1[:,0], hit_1[:,1], c=makeidx(hit_1), cmap='jet')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
hit_1.shape

(11, 2)

In [33]:
np.abs(hit_1[1:,0]-hit_1[:-1,0]).sum() + np.abs(hit_1[1:,1]-hit_1[:-1,1]).sum()

0.0002820268985573365

In [36]:
np.linalg.norm(hit_1[1:,:]-hit_1[:-1,:], axis=1).sum()

0.0002003532896521826

In [37]:
np.var(hit_1, axis=0).sum()

2.963103151171133e-08

In [16]:
rs = np.linspace(0,0.2, 50)
distsin = np.zeros_like(rs)
devs = np.zeros_like(rs)
for i, r in enumerate(rs):
    mu = np.real(r*mu2)
    distsin[i] = np.linalg.norm(mu[:2])
    ray0 = sys.screen.eigenvectors_to_rays(mu)
    traj_hit = sys.propagate(ray0, Nrt=30, at_screen=True)
    hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
    hit_1 = hit_scr[::3,:]
    devs[i] = np.var(hit_1, axis=0).sum()

In [17]:
plt.figure()
plt.plot(distsin, devs)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
def degeneracy_length_ray(cavfct, parname, r=0, scanrange=1e-3, La=None, **kwargs):
    def get_dev(l, r, arg):
        kwargs=arg.copy()
        kwargs.update({parname: l})
        elements = cavfct(**kwargs)
        sys = RaySystem(elements)
        system = sys.abcd
        mu1, mu2 = system.q
        waist = system.waist_at(0)[0] #FIX
        rmu = np.linalg.norm(np.real(mu1[:2]))
        mu = np.real(r*waist/rmu*mu1)#np.real(r*mu2)
        ray0 = sys.screen.eigenvectors_to_rays(mu)
        traj_hit = sys.propagate(ray0, Nrt=30, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
        hit_1 = hit_scr[::3,:]
        hit_2 = hit_scr[1::3,:]
        hit_3 = hit_scr[2::3,:]
        #return np.var(hit_1, axis=0).sum()
        dev = np.linalg.norm(hit_1[1:,:]-hit_1[:-1,:], axis=1).sum() + np.linalg.norm(hit_2[1:,:]-hit_2[:-1,:], axis=1).sum() + np.linalg.norm(hit_3[1:,:]-hit_3[:-1,:], axis=1).sum()
        return dev
    if La is None:
        La = inspect.signature(cavfct).parameters[parname].default
    extrargs = kwargs.copy()
    #extrargs.pop(parname)
    #extrargs.update({'r': r})
    res = minimize_scalar(get_dev, bounds=((1-scanrange)*La, (1+scanrange)*La), args=(r, extrargs), method='bounded')
    return res

In [56]:
degeneracy_length_ray(cavfct, parname, r=0.1, scanrange=1e-2, c4=2.9e-4, c6=1e-5)

     fun: 8.920091017142676e-08
 message: 'Solution found.'
    nfev: 14
  status: 0
 success: True
       x: 24.470980523601714

In [87]:
rs = np.linspace(np.finfo(np.float32).resolution,8., 100)
Ls = np.zeros_like(rs)
lcur = Ldeg
for i, r in enumerate(rs):
    try:
        l = degeneracy_length_ray(cavfct, parname, r=r, scanrange=1e-1, La=lcur, c4=3.02e-4, c6=1.97e-6).x
    except:
        Ls[i] = np.nan
    else:
        Ls[i] = l
        lcur = l

In [88]:
idx = np.where(rs>7.)[0][0] #only for up to
coef = np.polyfit(rs[:idx], Ls[:idx], deg=6)
coef

array([ 2.98732671e-09, -5.88683855e-08,  4.83483667e-07, -1.81820433e-06,
       -3.92308550e-05, -1.10138071e-06,  2.44709769e+01])

In [89]:
plt.figure()
plt.plot(rs, Ls)
plt.title('Degeneracy length vs radius')
plt.xlabel(r'$r_{in}$ [waists]')
plt.ylabel(r'$L_{deg}$ [mm]')
plt.axhline(Ldeg, color='grey')
plt.plot(rs, np.polyval(coef, rs), '--')
plt.plot(rs, np.polyval(coef[4:], rs), '-.')
plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
magnitude = lambda x: int(np.floor(np.log10(x)))

In [117]:
cavfct = LensCavAsphere
def get_dev(r, **kwargs):
    elements = cavfct(**kwargs)
    sys = RaySystem(elements)
    system = sys.abcd
    mu1, mu2 = system.q
    waist = system.waist_at(0)[0] #FIX
    rmu = np.linalg.norm(np.real(mu1[:2]))
    mu = np.real(r*waist/rmu*mu1)#np.real(r*mu2)
    ray0 = sys.screen.eigenvectors_to_rays(mu)
    traj_hit = sys.propagate(ray0, Nrt=300, at_screen=True)
    hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
    hit_1 = hit_scr[::3,:]
    hit_2 = hit_scr[1::3,:]
    hit_3 = hit_scr[2::3,:]
    #return np.var(hit_1, axis=0).sum()
    dev = np.linalg.norm(hit_1[1:,:]-hit_1[:-1,:], axis=1).sum() + np.linalg.norm(hit_2[1:,:]-hit_2[:-1,:], axis=1).sum() + np.linalg.norm(hit_3[1:,:]-hit_3[:-1,:], axis=1).sum()
    return dev

fig, ax = plt.subplots()
line = ax.plot([], [])[0]
plt.title('Deviation vs radius')
plt.xlabel(r'$r_{in}$ [waists]')
plt.ylabel(r'$dev$ [mm]')
plt.show()

def update(scalings, **kwargs):
    rs = np.linspace(np.finfo(np.float32).resolution,12., 100)
    Ls = np.zeros_like(rs)
    
    scargs = {}
    for k, v in kwargs.items():
        scargs[k] = v*10**scalings[k]
    
    for i, r in enumerate(rs):
        try:
            d = get_dev(r=r, **scargs)
        except:
            Ls[i] = np.nan
        else:
            Ls[i] = d
    line.set_xdata(rs)
    line.set_ydata(Ls)
    ax.set_xlim(rs[0], rs[-1])
    ax.set_ylim(1.1*np.nanmin(Ls), 1.1*np.nanmax(Ls))
    fig.canvas.draw_idle()
    
lo = Layout(width='80%', height='30px')
coefpars = {'c4': coeffc4(5.0), 'c6': coeffc6(5.0), 'c8': coeffc8(5.0)}
sliders = {}
scalings = {}
for k, v in coefpars.items():
    vs = v*10**(-magnitude(v))
    scalings[k] = magnitude(v)
    sliders[k] = FloatSlider(value=vs, min=-1.2*vs, max=1.2*vs, step=1e-3, readout_format='.3e', layout=lo)

print(scalings)
interactive(update, scalings=fixed(scalings), **sliders)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'c4': -3, 'c6': -5, 'c8': -7}


interactive(children=(FloatSlider(value=1.0, description='c4', layout=Layout(height='30px', width='80%'), max=…

In [119]:
def deviation_tot(x):
    rs = np.linspace(np.finfo(np.float32).resolution,8., 50)
    Ls = np.zeros_like(rs)
        
    for i, r in enumerate(rs):
        try:
            d = get_dev(r=r, c4=x[0], c6=x[1], c8=x[2])
        except:
            Ls[i] = np.nan
        else:
            Ls[i] = d
    return np.abs(np.nansum(Ls))

In [113]:
deviation_tot([3.02e-4, 1.97e-6, 5e-7])

6.172936758072929

In [120]:
res = minimize(deviation_tot, x0=[3.02e-4, -1.2e-6, 5e-7])
res

      fun: 5.800534762034565
 hess_inv: array([[ 1.07319174e-09, -1.35302795e-09,  4.32287762e-10],
       [-1.35302798e-09,  1.91213283e-09, -6.50931507e-10],
       [ 4.32287783e-10, -6.50931507e-10,  2.30625364e-10]])
      jac: array([-31.87669754,  11.53872949,  94.0566926 ])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 447
      nit: 5
     njev: 87
   status: 2
  success: False
        x: array([ 3.06671969e-04, -6.53668527e-06,  2.45530852e-06])

In [122]:
from scipy.optimize import differential_evolution

In [123]:
c4, c6, c8 = coeffc4(5.0), coeffc6(5.0), coeffc8(5.0)
bounds = [(-2*c4,2*c4), (-2*c6,2*c6), (-2*c8,2*c8)]
res2 = differential_evolution(deviation_tot, bounds)
res2

     fun: 5.807045234514024
     jac: array([   63.55558391,   255.8605372 , -6085.85530655])
 message: 'Optimization terminated successfully.'
    nfev: 923
     nit: 14
 success: True
       x: array([ 3.03930760e-04, -2.38444797e-06,  9.81242561e-07])

In [124]:
def plot_cs(**kwargs):
    rs = np.linspace(np.finfo(np.float32).resolution,12., 100)
    Ls = np.zeros_like(rs)
    
    scargs = {}
    for k, v in kwargs.items():
        scargs[k] = v*10**scalings[k]
    
    for i, r in enumerate(rs):
        try:
            d = get_dev(r=r, **scargs)
        except:
            Ls[i] = np.nan
        else:
            Ls[i] = d
    fig, ax = plt.subplots()
    line = ax.plot(rs, Ls)[0]
    plt.title('Deviation vs radius')
    plt.xlabel(r'$r_{in}$ [waists]')
    plt.ylabel(r'$dev$ [mm]')
    plt.show()

In [125]:
plot_cs(c4=3.03930760e-04, c6=-2.38444797e-06,  c8=9.81242561e-07)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
%load_ext snakeviz

In [63]:
#%snakeviz degeneracy_length_ray(cavfct, parname, r=3.1, scanrange=1e-2, c4=2.9e-4, c6=1e-5)

In [ ]:
plt.figure()
plt.plot(rs, Ls)
plt.title('Degeneracy length vs radius')
plt.xlabel(r'$r_{in}$ [waists]')
plt.ylabel(r'$L_{deg}$ [mm]')
plt.axhline(Ldeg, color='grey')
plt.plot(rs, np.polyval(coef, rs), '--')
plt.plot(rs, np.polyval(coef[4:], rs), '-.')
plt.show()

def update():
    rs = np.linspace(np.finfo(np.float32).resolution,8., 100)
    Ls = np.zeros_like(rs)
    lcur = Ldeg
    for i, r in enumerate(rs):
        try:
            l = degeneracy_length_ray(cavfct, parname, r=r, scanrange=1e-2, La=lcur, c4=2.9e-4, c6=1e-5).x
        except:
            Ls[i] = np.nan
        else:
            Ls[i] = l
            lcur = l
    idx = np.where(rs>7.)[0][0] #only for up to
    coef = np.polyfit(rs[:idx], Ls[:idx], deg=6)
    
        ars = FloatSlider(value=0.1, min=0., max=6., step=1e-2, readout_format='.3f', layout=lo)
    brs = FloatSlider(value=0.1, min=0., max=6., step=1e-2, readout_format='.3f', layout=lo)

In [21]:
def degeneracy_par_ray(cavfct, parname, r=0, scanrange=1e-3, p0=None):
    def get_dev(p, r):
        elements = cavfct(**{parname: p})
        sys = RaySystem(elements)
        system = sys.abcd
        mu1, mu2 = system.q
        waist = waist0 #system.waist_at(0)[0] FIX
        rmu = np.linalg.norm(np.real(mu2[:2]))
        mu = np.real(r*waist/rmu*mu2)#np.real(r*mu2)
        ray0 = sys.screen.eigenvectors_to_rays(mu)
        traj_hit = sys.propagate(ray0, Nrt=30, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
        hit_1 = hit_scr[::3,:]
        return np.var(hit_1, axis=0).sum()
    if p0 is None:
        p0 = inspect.signature(cavfct).parameters[parname].default
    if not isinstance(scanrange, tuple):
        scanrange = ((1-scanrange)*p0, (1+scanrange)*p0)
    res = minimize_scalar(get_dev, bounds=scanrange, args=(r), method='bounded')
    return res

In [22]:
rs = np.linspace(np.finfo(np.float32).resolution,8., 100)
Ps = np.zeros_like(rs)
pcur = 0.
for i, r in enumerate(rs):
    try:
        p = degeneracy_par_ray(cavfct, 'c4', r=r, scanrange=(0., 1e-2), p0=pcur).x
    except:
        Ps[i] = np.nan
    else:
        Ps[i] = p
        #pcur = p

In [23]:
plt.figure()
plt.plot(rs, Ps)
plt.title('Optimal C4 coefficient vs radius')
plt.xlabel(r'$r_{in}$ [waists]')
plt.ylabel(r'$L_{deg}$ [mm]')
#plt.axhline(Ldeg, color='grey')
#plt.plot(rs, np.polyval(coef, rs), '--')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
def plot_config(l=0., ar=0.0, br=0.0, ap=0., bp=0., Nrt=500, ms = 8., **kwargs):
        fig, (ax, bx, cx) = plt.subplots(ncols=3, figsize=(11,3.3), sharex=True, sharey=True)
        linea = ax.scatter([], [], c=[], cmap='jet', s=ms)
        linec = cx.scatter([], [], c=[], cmap='jet', s=ms)
        lineb = bx.scatter([], [], c=[], cmap='jet', s=ms)
        ax.set_title('Raytracing')
        bx.set_title('ABCD')
        cx.set_title('ABCD (Extracted)')
        plt.xlabel('x [mm]')
        plt.ylabel('y [mm]')
        kwargs.update({parname: l})
        elements = cavfct(**kwargs)
        sys = RaySystem(elements)
        system = sys.abcd
        mu1, mu2 = system.q
        waist = system.waist_at(0)[0]
        rmu1 = np.linalg.norm(np.real(mu1[:2]))
        rmu2 = np.linalg.norm(np.real(mu2[:2]))
        mu = np.real(ar*waist/rmu1*mu1*np.exp(1j*ap) + br*waist/rmu2*mu2*np.exp(1j*bp))
        ray0 = sys.screen.eigenvectors_to_rays(mu)
 
        traj_hit = sys.propagate(ray0, Nrt=Nrt, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])

        linea.set_offsets(hit_scr[:,:])
        linea.set_array(makeidx(hit_scr))
        
        hit_m = propagate_ABCD(mu, system.abcd_rt, Nrt=Nrt)
        lineb.set_offsets(hit_m[:,0:2])
        lineb.set_array(makeidx(hit_m))
        
        #abcd_fd = sys.extract_ABCD(epsr=1e-4, epss=1e-4, Nrt=1) 
        #hit_fd = propagate_ABCD(mu, abcd_fd, Nrt=Nrt)
        #linec.set_offsets(hit_fd[:,0:2])
        #linec.set_array(makeidx(hit_fd))
        
        linea.set_clim(vmin=0, vmax=Nrt)
        lineb.set_clim(vmin=0, vmax=Nrt)
        linec.set_clim(vmin=0, vmax=Nrt)
            
        ax.set_xlim(1.5*np.nanmin(hit_scr[:,0]), 1.5*np.nanmax(hit_scr[:,0]))
        ax.set_ylim(1.5*np.nanmin(hit_scr[:,1]), 1.5*np.nanmax(hit_scr[:,1]))
        return fig

In [26]:
rw = 5.
interp=True
if interp:
    lw = np.polyval(coef, rw)
else:
    idx = np.where(rs>rw)[0][0]
    lw = Ls[idx]
    rw = rs[idx]
f = plot_config(l=lw, br=rw, Nrt=500, ms=4, c4=0.416e-2)
f.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [117]:
plt.close("all")

## Eigenray

In [65]:
def plot_config_ray(mu, l=0., Nrt=500, ms = 8., **kwargs):
        fig, (ax, bx, cx) = plt.subplots(ncols=3, figsize=(11,3.3), sharex=True, sharey=True)
        linea = ax.scatter([], [], c=[], cmap='jet', s=ms)
        linec = cx.scatter([], [], c=[], cmap='jet', s=ms)
        lineb = bx.scatter([], [], c=[], cmap='jet', s=ms)
        ax.set_title('Raytracing')
        bx.set_title('ABCD')
        cx.set_title('ABCD (Extracted)')
        plt.xlabel('x [mm]')
        plt.ylabel('y [mm]')
        elements = cavfct(**{parname: l})
        sys = RaySystem(elements)
        ray0 = sys.screen.eigenvectors_to_rays(mu)

        traj_hit = sys.propagate(ray0, Nrt=Nrt, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])

        linea.set_offsets(hit_scr[:,:])
        linea.set_array(makeidx(hit_scr))
        
        hit_m = propagate_ABCD(mu, system.abcd_rt, Nrt=Nrt)
        lineb.set_offsets(hit_m[:,0:2])
        lineb.set_array(makeidx(hit_m))
        
        abcd_fd = sys.extract_ABCD(epsr=1e-4, epss=1e-4, Nrt=1) 
        hit_fd = propagate_ABCD(mu, abcd_fd, Nrt=Nrt)
        linec.set_offsets(hit_fd[:,0:2])
        linec.set_array(makeidx(hit_fd))
        
        linea.set_clim(vmin=0, vmax=Nrt)
        lineb.set_clim(vmin=0, vmax=Nrt)
        linec.set_clim(vmin=0, vmax=Nrt)
            
        ax.set_xlim(1.5*np.nanmin(hit_scr[:,0]), 1.5*np.nanmax(hit_scr[:,0]))
        ax.set_ylim(1.5*np.nanmin(hit_scr[:,1]), 1.5*np.nanmax(hit_scr[:,1]))
        return fig

In [74]:
#mu = np.array([5e-3, 0., 0, 0])
system = sys.abcd
mu1, mu2 = system.q
waist = system.waist_at(0)[0]
rmu = np.linalg.norm(np.real(mu2[:2]))
r = 5e-3
mu = np.real(r*waist/rmu*mu2)
ray0 = sys.screen.eigenvectors_to_rays(mu)
lr = np.zeros((2, 1, 3))
#lr[1,:,:] = 0.05
lr = 0.05
#reig = sys.find_eigenray(ray0, lr=lr, Nrt=30, debug=True)
reig = sys.find_eigenray_mpe(ray0, lr=lr, Nrt=30, debug=True)
mueig = sys.screen.rays_to_eigenvectors(reig)
mueig

Finished in 49 steps, reached tol 5.473e-05
Finished in 49 steps, reached tol 2.166e-09
Finished in 49 steps, reached tol 4.613e-08
Finished in 49 steps, reached tol 3.327e-08
Finished in 49 steps, reached tol 3.617e-08


array([[ 6.07047033e-05,  7.65926464e-05, -1.10603097e-04,
        -3.29846840e-04]])

In [75]:
f = plot_config_ray(np.squeeze(mueig), Ldeg)
f.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
def degeneracy_length_eigenray(cavfct, parname, r=0, scanrange=1e-3, La=None):
    def get_dev(l, r):
        elements = cavfct(**{parname: l})
        sys = RaySystem(elements)
        system = sys.abcd
        mu1, mu2 = system.q
        waist = system.waist_at(0)[0]
        rmu = np.linalg.norm(np.real(mu2[:2]))
        mu = np.real(r*waist/rmu*mu2)
        ray0 = sys.screen.eigenvectors_to_rays(mu)
        lr = 0.05
        reig = sys.find_eigenray(ray0, lr=lr, Nrt=3, debug=False)
        #reig = sys.find_eigenray_mpe(ray0, lr=lr, Nrt=30, debug=False)
        mueig = sys.screen.rays_to_eigenvectors(reig)
        
        ray1 = sys.screen.eigenvectors_to_rays(mu)
        traj_hit = sys.propagate(ray1, Nrt=30, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
        hit_1 = hit_scr[::3,:]
        return np.var(hit_1, axis=0).sum()
    if La is None:
        La = inspect.signature(cavfct).parameters[parname].default
    res = minimize_scalar(get_dev, bounds=((1-scanrange)*La, (1+scanrange)*La), args=(r,), method='bounded')
    return res

In [82]:
degeneracy_length_eigenray(cavfct, parname, r=1e-3, scanrange=1e-3, La=Ldeg)

     fun: 2.020474060964267e-08
 message: 'Solution found.'
    nfev: 10
  status: 0
 success: True
       x: 21.506287396753475

In [83]:
rs = np.linspace(0,8., 100)
Ls = np.zeros_like(rs)
lcur = Ldeg
for i, r in enumerate(rs):
    try:
        l = degeneracy_length_eigenray(cavfct, parname, r=r, scanrange=1e-2, La=lcur).x
    except:
        Ls[i] = np.nan
    else:
        Ls[i] = l
        lcur = l

In [84]:
idx = np.where(rs>5.)[0][0] #only for up to
coef = np.polyfit(rs[:idx], Ls[:idx], deg=4)
coef

array([-2.43816029e-04,  2.60880858e-03, -2.34249747e-02,  1.36223586e-02,
        2.15193646e+01])

In [85]:
plt.figure()
plt.plot(rs, Ls)
plt.title('Degeneracy length vs radius')
plt.xlabel(r'$r_{in}$ [waists]')
plt.ylabel(r'$L_{deg}$ [mm]')
plt.axhline(Ldeg, color='grey')
plt.plot(rs, np.polyval(coef, rs), '--')
plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
rw = 5.5
interp=False
if interp:
    lw = np.polyval(coef, rw)
else:
    idx = np.where(rs>rw)[0][0]
    lw = Ls[idx]
    rw = rs[idx]
f = plot_config(l=lw, br=rw, Nrt=500, ms=4)
f.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …